In [11]:
%pip install numpy
%pip install scipy
%pip install scikit-learn
%pip install pandas
%pip install boto3
%pip install joblib
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from itertools import chain
import scipy.sparse._csr
import pandas as pd
import re
import nltk
import joblib
import boto3
import os
import nltk

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

ModuleNotFoundError: No module named 'scipy.sparse._csr'

In [3]:
stop_words = set(stopwords.words('english'))
wnet = nltk.WordNetLemmatizer()

In [4]:
def read_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

In [5]:
genres = [
    'Country',
    'Rap',
    'Rock'
]

In [6]:
bucket_name = 'ml-models-sprint2'
s3_client = boto3.client('s3')

In [10]:
loaded_models = {}
for genre in genres:
    filename = f'{genre}_model.joblib'
    print(filename)
    loaded_models[genre] = joblib.load(filename)

Country_model.joblib


KeyError: 255

In [13]:
vectorize = joblib.load('tfidf_vectorizer.joblib')

ModuleNotFoundError: No module named 'scipy.sparse._csr'

In [ ]:
print(loaded_models)
print(vectorize)

In [ ]:
file_key = 'lyric.txt'

response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
text_content = response['Body'].read().decode('utf-8')

df = pd.DataFrame({'lyrics': [text_content]})
df

In [ ]:
tokenized = [word_tokenize(lyr) for lyr in df['lyrics'].astype(str)]
stop_vec = [[w for w in tok if w not in stop_words] for tok in tokenized]
clean_vec = [[word for word in lyr if word.isalpha()] for lyr in stop_vec]
lem = [[wnet.lemmatize(w) for w in lyr] for lyr in clean_vec]

In [ ]:
lyrics_tay = [' '.join(lyr) for lyr in lem]
single_entry = vectorize.transform(lyrics_tay)
s_e = single_entry.todense().tolist()

probabilities = {genre: loaded_models[genre].predict_proba(s_e)[0][1] for genre in genres}

In [ ]:
for genre, prob in probabilities.items():
    print(f"{genre}: {prob * 100:.2f}%")